# 다대다 병합

### 클리브랜드 미술관 소장품 데이터 사용
- 소장품 매체 인용
- 소장품별 작가 정보

In [1]:
import pandas as pd
cmacitations = pd.read_csv('data/cmacitations.csv')
cmacreators = pd.read_csv('data/cmacreators.csv')

In [2]:
cmacitations.head()

,id,citation
0,92937,"Milliken, William M. ""The Second Exhibition of..."
1,92937,"Glasier, Jessie C. ""Museum Gets Prize-Winning ..."
2,92937,"""Cleveland Museum Acquires Typical Pictures by..."
3,92937,"Milliken, William M. ""Two Examples of Modern P..."
4,92937,<em>Memorial Exhibition of the Work of George ...


In [3]:
cmacitations.shape

(11642, 2)

In [5]:
cmacitations.id.nunique()

935

In [6]:
cmacreators.loc[:, ['id', 'creator', 'birth_year']].head(10)

,id,creator,birth_year
0,92937,"George Bellows (American, 1882-1925)",1882
1,94979,"John Singleton Copley (American, 1738-1815)",1738
2,137259,"Gustave Courbet (French, 1819-1877)",1819
3,141639,"Frederic Edwin Church (American, 1826-1900)",1826
4,93014,"Thomas Cole (American, 1801-1848)",1801
5,110180,"Albert Pinkham Ryder (American, 1847-1917)",1847
6,135299,"Vincent van Gogh (Dutch, 1853-1890)",1853
7,125249,"Vincent van Gogh (Dutch, 1853-1890)",1853
8,126769,"Henri Rousseau (French, 1844-1910)",1844
9,135382,"Claude Monet (French, 1840-1926)",1840


In [7]:
cmacreators.shape

(737, 8)

In [8]:
cmacreators.id.nunique()

654

### citations 데이터에서 병합 기준 값의 중복 있음
- 예를 들어, 소장품 148758에 대해 174회의 매체 인용이 있었음

In [10]:
cmacitations.id.value_counts().head(10)

148758    174
122351    116
92937      98
123168     94
94979      93
149112     93
124245     87
128842     86
102578     84
93014      79
Name: id, dtype: int64

### creators 데이터에 병합 기준 값의 중복 있음

In [12]:
cmacreators.id.value_counts().head(10)

149386    4
140001    4
114537    3
146797    3
149041    3
142753    3
114538    3
140427    3
93173     3
142752    3
Name: id, dtype: int64

### 병합을 확인
- checkmerge 함수 사용

In [13]:
def checkmerge(dfleft, dfright, idvar):
    dfleft['inleft'] = "Y"
    dfright['inright'] = "Y"
    dfboth = pd.merge(dfleft[[idvar, 'inleft']], dfright[[idvar, 'inright']],
                     on=[idvar], how='outer')
    dfboth.fillna('N', inplace=True)
    print(pd.crosstab(dfboth.inleft, dfboth.inright))

In [14]:
checkmerge(cmacitations.copy(), cmacreators.copy(), 'id')

inright     N     Y
inleft             
N           0    46
Y        2579  9701


### 양쪽 데이터프레임에 중복된 병합 기준 값 확인

In [15]:
cmacitations.loc[cmacitations.id==124733]

,id,citation
8963,124733,"Weigel, J. A. G. <em>Catalog einer Sammlung vo..."
8964,124733,"Winkler, Friedrich. <em>Die Zeichnungen Albrec..."
8965,124733,"Francis, Henry S. ""Drawing of a Dead Blue Jay ..."
8966,124733,"Kurz, Otto. <em>Fakes: A Handbook for Collecto..."
8967,124733,Minneapolis Institute of Arts. <em>Watercolors...
8968,124733,"Pilz, Kurt. ""Hans Hoffmann: Ein Nürnberger Dür..."
8969,124733,"Koschatzky, Walter and Alice Strobl. <em>Düre..."
8970,124733,"Johnson, Mark M<em>. Idea to Image: Preparator..."
8971,124733,"Kaufmann, Thomas DaCosta. <em>Drawings from th..."
8972,124733,"Koreny, Fritz. <em>Albrecht Dürer and the ani..."


In [16]:
cmacreators.loc[cmacreators.id==124733, ['id', 'creator', 'birth_year', 'title']]

,id,creator,birth_year,title
449,124733,"Albrecht Dürer (German, 1471-1528)",1471,Dead Blue Roller
450,124733,"Hans Hoffmann (German, 1545/50-1591/92)",1545/50,Dead Blue Roller


### 다대다 병합

In [17]:
cma = pd.merge(cmacitations, cmacreators, on=['id'], how='outer')
cma['citation'] = cma.citation.str[0:20]
cma['creator'] = cma.creator.str[0:20]

In [18]:
cma.loc[cma.id==124733, ['citation', 'creator','birth_year']]

,citation,creator,birth_year
9457,"Weigel, J. A. G. <em",Albrecht Dürer (Germ,1471
9458,"Weigel, J. A. G. <em",Hans Hoffmann (Germa,1545/50
9459,"Winkler, Friedrich.",Albrecht Dürer (Germ,1471
9460,"Winkler, Friedrich.",Hans Hoffmann (Germa,1545/50
9461,"Francis, Henry S. ""D",Albrecht Dürer (Germ,1471
9462,"Francis, Henry S. ""D",Hans Hoffmann (Germa,1545/50
9463,"Kurz, Otto. <em>Fake",Albrecht Dürer (Germ,1471
9464,"Kurz, Otto. <em>Fake",Hans Hoffmann (Germa,1545/50
9465,Minneapolis Institut,Albrecht Dürer (Germ,1471
9466,Minneapolis Institut,Hans Hoffmann (Germa,1545/50
